---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---

# NBJ 1 — Où l’IA regarde ?
### Initiation à l'explicabilité en vision par ordinateur

**Objectif :** comprendre comment une IA "regarde" une image et quelles zones influencent sa décision.

Dans ce premier TP, nous allons :
- afficher une image et demander à un modèle ce qu’il voit ;
- modifier l’image petit morceau par petit morceau (occlusion) ;
- calculer l’importance des pixels à l’aide d’un gradient simple (saliency) ;
- comparer les différentes explications ;
- réfléchir à la notion même d’“explicabilité”.

Aucun outil XAI externe : tout est fait à la main, pour comprendre les bases.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import tensorflow as tf

In [ ]:
# TODO : placer une image adaptée dans le même dossier, par exemple 'image_chien.jpg'
img_path = "image_chien.jpg"  # à adapter

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_input = np.expand_dims(img_array.copy(), axis=0)
img_preprocessed = preprocess_input(img_input)

plt.imshow(img_array.astype("uint8"))
plt.title("Image originale")
plt.axis("off")

### ❓ Question 1  
Selon toi, **quelles zones** de cette image l'IA va-t-elle regarder pour décider ce que c'est ?  
Explique ton intuition.

In [ ]:
model = mobilenet_v2.MobileNetV2(weights="imagenet")
preds = model.predict(img_preprocessed)
decode_predictions(preds, top=3)[0]

### ❓ Questions 2–3  
2. Le modèle a-t-il raison ? Pourquoi ?  
3. D’après toi, **où** regarde-t-il pour prendre sa décision ?

In [ ]:
def occlusion_map(model, img_array, patch_size=30, stride=10):
    h, w, _ = img_array.shape
    heatmap = np.zeros((h, w), dtype=np.float32)

    base_pred = model.predict(preprocess_input(np.expand_dims(img_array, 0)))[0]
    base_class = np.argmax(base_pred)

    for y in range(0, h - patch_size, stride):
        for x in range(0, w - patch_size, stride):
            occluded = img_array.copy()
            occluded[y:y+patch_size, x:x+patch_size] = 0

            pred = model.predict(preprocess_input(np.expand_dims(occluded, 0)))[0]
            drop = float(base_pred[base_class] - pred[base_class])
            heatmap[y:y+patch_size, x:x+patch_size] = drop

    heatmap_norm = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)
    return heatmap_norm

In [ ]:
heatmap_occ = occlusion_map(model, img_array, patch_size=30, stride=10)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(img_array.astype("uint8"))
plt.axis("off")
plt.title("Image originale")

plt.subplot(1,2,2)
plt.imshow(heatmap_occ, cmap="jet")
plt.axis("off")
plt.title("Occlusion — zones importantes")
plt.tight_layout()

### ❓ Questions 4–6  
4. Quand on cache une petite zone, comment la prédiction change-t-elle ?  
5. Quelles zones semblent les plus importantes pour le modèle ?  
6. L’IA regarde-t-elle les mêmes zones que toi ? Pourquoi ?

In [ ]:
img_tf = tf.convert_to_tensor(img_preprocessed, dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch(img_tf)
    preds_tf = model(img_tf)
    top_index = tf.argmax(preds_tf[0])

grads = tape.gradient(preds_tf[0, top_index], img_tf)[0]
saliency = np.max(np.abs(grads.numpy()), axis=-1)

saliency_norm = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-8)

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(img_array.astype("uint8"))
plt.axis("off")
plt.title("Image originale")

plt.subplot(1,2,2)
plt.imshow(saliency_norm, cmap="inferno")
plt.axis("off")
plt.title("Saliency Map (gradient)")
plt.tight_layout()

### ❓ Questions 7–9  
7. Les pixels les plus brillants correspondent-ils aux zones importantes ?  
8. L’explication est-elle “propre” ou bruitée ?  
9. Quelles différences vois-tu entre l’occlusion et la saliency ?

In [ ]:
def smoothgrad(model, img_preprocessed, n_samples=20, noise=0.2):
    grads_list = []

    for _ in range(n_samples):
        noisy = img_preprocessed + noise * np.random.normal(size=img_preprocessed.shape)
        noisy_tf = tf.convert_to_tensor(noisy, dtype=tf.float32)

        with tf.GradientTape() as tape:
            tape.watch(noisy_tf)
            preds_tf = model(noisy_tf)
            top_index = tf.argmax(preds_tf[0])

        g = tape.gradient(preds_tf[0, top_index], noisy_tf)[0].numpy()
        grads_list.append(np.max(np.abs(g), axis=-1))

    avg = np.mean(np.stack(grads_list, axis=0), axis=0)
    avg_norm = (avg - avg.min()) / (avg.max() - avg.min() + 1e-8)
    return avg_norm

smooth = smoothgrad(model, img_preprocessed)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.imshow(saliency_norm, cmap='inferno')
plt.title("Saliency brute")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(smooth, cmap='inferno')
plt.title("SmoothGrad")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(heatmap_occ, cmap='jet')
plt.title("Occlusion")
plt.axis("off")

plt.tight_layout()

### ❓ Questions 10–12  
10. SmoothGrad est-il plus lisible que la saliency brute ? Pourquoi ?  
11. Les zones importantes sont-elles cohérentes entre les 3 méthodes ?  
12. Laquelle te paraît la plus intuitive ? Pourquoi ?

# 🧠 Synthèse — Qu'avons-nous appris ?

- Une IA ne regarde **pas toute l’image** : elle se concentre sur quelques zones clés.  
- L’occlusion permet de voir **où** une zone influence la décision.  
- Les saliency maps montrent **quels pixels** ont le plus d’impact.  
- SmoothGrad améliore la lisibilité.  
- Ces explications nous aident à comprendre *comment* l’IA prend ses décisions.

### ❓ Question finale  
En une phrase : **pour toi, qu’est-ce qu’une “bonne explication” d’un modèle IA ?**

---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---